In [ ]:
from aoc.utils import download_input

In [ ]:
file_path = download_input(day=9, year=2024, output_dir="input_files")

In [ ]:
def parse_disk_map(disk_map):
    """Parse the dense disk map into a list of blocks."""
    blocks = []
    file_id = 0
    is_file = True  # Alternates between file and free space
    for digit in map(int, disk_map):
        if is_file:
            blocks.extend([file_id] * digit)
            file_id += 1
        else:
            blocks.extend(["."] * digit)  # Represent free space as "."
        is_file = not is_file
    return blocks


def compact_disk(blocks):
    """Compact the disk by moving file blocks into free space."""
    for i in range(len(blocks) - 1, -1, -1):  # Start from the end of the disk
        if blocks[i] == ".":
            continue
        for j in range(i):
            if blocks[j] == ".":
                # Move the file block to the leftmost free space
                blocks[j], blocks[i] = blocks[i], "."
                break
    return blocks


def calculate_checksum(blocks):
    """Calculate the filesystem checksum."""
    checksum = 0
    for position, block in enumerate(blocks):
        if block != ".":
            checksum += position * block
    return checksum


def part1(disk_map):
    """Solve the disk fragmenter problem."""
    # Parse the disk map into blocks
    blocks = parse_disk_map(disk_map)

    # Compact the disk
    compacted_blocks = compact_disk(blocks)

    # Calculate the checksum
    return calculate_checksum(compacted_blocks)

In [ ]:
# Example input
example_disk_map = "2333133121414131402"

# Solve the example
checksum = part1(example_disk_map)
print(f"Filesystem checksum: {checksum}")

In [ ]:
# Read the file content
with open(file_path, "r") as file:
    actual_disk_map = file.read().strip()
    
checksum = part1(actual_disk_map)
print(f"Filesystem checksum: {checksum}")

In [ ]:
def find_free_span(blocks, file_loc, file_size):
    """Find the leftmost span of free space that can fit the file."""
    start = 0
    while start < file_loc:
        # Count the size of the contiguous free space
        span_size = 0
        while start + span_size < file_loc and blocks[start + span_size] == ".":
            span_size += 1
        
        # If the span is large enough, return its starting position
        if span_size >= file_size:
            return start
        
        # Move to the end of the current span
        start += max(span_size, 1)
    return None  # No suitable span found


def compact_disk_by_files(blocks):
    """Compact the disk by moving whole files."""
    file_positions = {}  # Track file positions and sizes
    for i, block in enumerate(blocks):
        if block != ".":
            file_positions.setdefault(block, []).append(i)

    # Sort files by file ID in descending order
    for file_id in sorted(file_positions.keys(), reverse=True):
        file_blocks = file_positions[file_id]
        file_size = len(file_blocks)
        free_span_start = find_free_span(blocks, file_blocks[0], file_size)

        if free_span_start is not None:
            # Move the file to the free span
            for i in file_blocks:
                blocks[i] = "."  # Clear the original blocks
            for i in range(file_size):
                blocks[free_span_start + i] = file_id

    return blocks


def part2(disk_map):
    """Solve the disk fragmenter problem."""
    # Parse the disk map into blocks
    blocks = parse_disk_map(disk_map)

    # Compact the disk
    compacted_blocks = compact_disk_by_files(blocks)

    # Calculate the checksum
    return calculate_checksum(compacted_blocks)

In [ ]:
# Solve the example
checksum = part2(example_disk_map)
print(f"Filesystem checksum: {checksum}")

In [ ]:
checksum = part2(actual_disk_map)
print(f"Filesystem checksum: {checksum}")